<a href="https://colab.research.google.com/github/itsPronay/HSIC/blob/main/MViT_batch_size_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!curl -L -o pavia-university-hsi.zip\
  https://www.kaggle.com/api/v1/datasets/download/syamkakarla/pavia-university-hsi

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 33.2M  100 33.2M    0     0  7124k      0  0:00:04  0:00:04 --:--:-- 9632k


In [3]:
!git clone https://github.com/swiftest/MViT.git

Cloning into 'MViT'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 98 (delta 28), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 2.59 MiB | 14.92 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [4]:
!unzip pavia-university-hsi.zip

Archive:  pavia-university-hsi.zip
  inflating: PaviaU.mat              
  inflating: PaviaU_gt.mat           


In [5]:
import argparse

parser = argparse.ArgumentParser('MViT argparser')

parser.add_argument('--patch_size', type=int, default=15)

args, _ = parser.parse_known_args()

In [6]:
import scipy.io as sio
import os

In [7]:
cwd = os.getcwd()
data = sio.loadmat(os.path.join(cwd, 'PaviaU.mat'))
ground_truth = sio.loadmat(os.path.join(cwd, 'PaviaU_gt.mat'))

In [8]:
print(data.keys())
print(ground_truth.keys())

dict_keys(['__header__', '__version__', '__globals__', 'paviaU'])
dict_keys(['__header__', '__version__', '__globals__', 'paviaU_gt'])


In [9]:
data = data['paviaU']
ground_truth = ground_truth['paviaU_gt']

In [10]:
print(data.shape)
print(ground_truth.shape)

(610, 340, 103)
(610, 340)


In [11]:
import numpy as np

print('Number of class : {}'.format(np.max(ground_truth)))

Number of class : 9


In [12]:
# normalize data
from sklearn.preprocessing import StandardScaler

height, width, band = data.shape

data = data.reshape(height * width , band)
scaler = StandardScaler()
data = scaler.fit_transform(data)
data = data.reshape(height, width, band)
print('Rshaped back into : {}'.format(data.shape))

Rshaped back into : (610, 340, 103)


In [13]:
# apply pca
from sklearn.decomposition import PCA

def applyPCA(data, numComponents=30):
    new_data = np.reshape(data, (-1, data.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    new_data = pca.fit_transform(new_data)
    new_data = np.reshape(new_data, (data.shape[0], data.shape[1], numComponents))
    return new_data, pca

In [14]:
K = 30
data, pca = applyPCA(data, K)
print(data.shape)

(610, 340, 30)


In [15]:
 height, width, band = data.shape
 print('Height: {}, width: {}, band: {}'.format(height, width, band))

Height: 610, width: 340, band: 30


In [16]:
from MViT.data_prepare import mirror_hsi

data = mirror_hsi(height, width, band, data, patch_size=args.patch_size)

*******************************************************
patch_size : 15
mirror_data shape : [624, 354, 30]
*******************************************************


In [17]:
ground_truth.shape

(610, 340)

If ground truth is
```
ground_truth = [
  [0, 1, 2, 1],
  [2, 0, 1, 3],
  [1, 3, 0, 2],
  [3, 1, 2, 0]
]
```

and i is 1,
`each_class = np.argwhere(ground_truth == i + 1)` will give us
```
 [[0,1],   → row 0, col 1
  [0,3],   → row 0, col 3
  [1,2],   → row 1, col 2
  [2,0],   → row 2, col 0
  [3,1]]   → row 3, col 1
```
then `each_class.shape` would be `(5, 2)`
so, `each_class.shape[0] = 5`

in `number_train`, we just append the shape

`number_tran.append(each_class.shape[0])`
and in `pos_train` dict we will insert `each_class` with i (class_number)

-----------------------------
then in the second part, we take the first value of `pos_train` and store in `total_pos_train`


then we run a loop from 1 to num_class (1 to 8) in case of pavia

and here
```
   total_pos_train = np.r_[total_pos_train, pos_train[i]]
```
we just keep adding it like this
```
# [[0,1],
#  [0,3],
#  [1,2],   ← class 1
#  [0,2],
#  [1,0],   ← class 2
#  [1,3],
#  [2,1]]   ← class 3 added
```
then convert its type

and then return it

## What it does?
it answers for each pixel in the image, WHERE is it and WHAT class does it belong to?

```
total_pos_train = [
  [0,1],  ← class 1... but how do you know?
  [0,3],  ← class 1
  [1,2],  ← class 1
  [0,2],  ← class 2... but how do you know?
  [1,0],  ← class 2
  [1,3],  ← class 3... but how do you know?
]
```

How does it know?
thats where `number_train` comes in — it's used separately to build labels:
```
number_train = [3, 2, 1]  # class1 has 3, class2 has 2, class3 has 1
```


In [18]:
def choose_train_and_test(ground_truth, seed=42):
  num_classes = np.max(ground_truth)
  number_train = []
  pos_train = {}

  for i in range(num_classes):
    each_class = np.argwhere(ground_truth == i + 1)
    total_samples = each_class.shape[0]
    number_train.append(total_samples)
    print('Class {} has {} samples'.format(i + 1, total_samples))
    pos_train[i] = each_class

  total_pos_train = pos_train[0]
  for i in range(1, num_classes):
    total_pos_train = np.r_[total_pos_train, pos_train[i]]
  total_pos_train = total_pos_train.astype(int)

  return total_pos_train, number_train


In [19]:
total_pos_train, number_train = choose_train_and_test(ground_truth)

Class 1 has 6631 samples
Class 2 has 18649 samples
Class 3 has 2099 samples
Class 4 has 3064 samples
Class 5 has 1345 samples
Class 6 has 5029 samples
Class 7 has 1330 samples
Class 8 has 3682 samples
Class 9 has 947 samples


In [20]:
print(total_pos_train.shape)

(42776, 2)


In [21]:
def gain_neighborhood_pixel(mirror_data, pos, i, patch_size):
    x = pos[i, 0]
    y = pos[i, 1]
    temp_image = mirror_data[x:(x+patch_size), y:(y+patch_size), :]
    return temp_image

In [22]:
def train_data(mirror_data, band, train_position, patch_size=15):
  train_data = np.zeros((train_position.shape[0], patch_size, patch_size, band))

  for i in range(train_position.shape[0]):
      train_data[i] = gain_neighborhood_pixel(mirror_data, train_position, i, patch_size)

  return train_data



```
number_train = [5, 4, 3]  # class1 has 5 pixels, class2 has 4, class3 has 3
```

The Loop
```
for i in range(num_classes):       # for each class
    for j in range(number_train[i]): # repeat i, that many times
        y_train.append(i)
```

```
i=0 (class 1) → append 0, five times  → [0, 0, 0, 0, 0]
i=1 (class 2) → append 1, four times  → [0, 0, 0, 0, 0, 1, 1, 1, 1]
i=2 (class 3) → append 2, three times → [0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2]
```


In [23]:
def train_and_test_label(number_train, num_classes):
    y_train = []
    for i in range(num_classes):
        for j in range(number_train[i]):
            y_train.append(i)
    y_train = np.array(y_train)
    print("y_train: shape = {}, type = {}".format(y_train.shape, y_train.dtype))
    print("*******************************************************")
    return y_train

In [24]:
x_train = train_data(data, band, total_pos_train)
y_train = train_and_test_label(number_train, 9)

y_train: shape = (42776,), type = int64
*******************************************************


In [25]:
print(x_train.shape)
print(y_train.shape)

(42776, 15, 15, 30)
(42776,)


In [26]:
transposed = x_train.transpose(0, 3, 1, 2)
print(transposed.shape)

(42776, 30, 15, 15)


In [27]:
import torch
loaded_torch = torch.from_numpy(transposed)
print(loaded_torch.shape)
# unsqueeze(1)
# print(un.shape)

torch.Size([42776, 30, 15, 15])


In [28]:
x_train = loaded_torch.unsqueeze(1).type(torch.FloatTensor)
print(x_train.shape)

torch.Size([42776, 1, 30, 15, 15])


In [29]:
y_train = torch.from_numpy(y_train).type(torch.LongTensor)

In [30]:
import torch.utils.data as Data

train_label = Data.TensorDataset(x_train, y_train)
train_loader = Data.DataLoader(train_label, batch_size=30, shuffle=True)

# Model

In [31]:
import torch.nn as nn
from einops import rearrange, repeat
from timm.models.vision_transformer import Block

def get_1d_sincos_pos_embed_from_grid(embed_dim, pos):
    """
    embed_dim: output dimension for each position
    pos: a list of positions to be encoded: size (M, )
    out: (M, D)
    """
    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=np.float32)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega  # (D/2, )  32

    pos = pos.reshape(-1)  # (M, )  169
    out = np.einsum('m, d -> md', pos, omega)  # (M, D/2), outer product

    emb_sin = np.sin(out)  # (M, D/2)
    emb_cos = np.cos(out) # (M, D/2)

    emb = np.concatenate([emb_sin, emb_cos], axis=1)  # (M, D)
    return emb


def get_2d_sincos_pos_embed_from_grid(embed_dim, grid):
    assert embed_dim % 2 == 0

    # use half of dimensions to encode grid_h
    emb_h = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[0])  # (H*W, D/2)
    emb_w = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[1])  # (H*W, D/2)

    emb = np.concatenate([emb_h, emb_w], axis=1)  # (H*W, D)
    return emb


def get_2d_sincos_pos_embed(embed_dim, grid_size, cls_token=True):
    """
    grid_size: int of the grid height and width
    return:
    pos_embed: [grid_size*grid_size, embed_dim] or [1+grid_size*grid_size, embed_dim] (w/ or w/o cls_token)
    """
    grid_h = np.arange(grid_size, dtype=np.float32)
    grid_w = np.arange(grid_size, dtype=np.float32)
    grid = np.meshgrid(grid_w, grid_h)  # here w goes first
    grid = np.stack(grid, axis=0)

    grid = grid.reshape([2, 1, grid_size, grid_size])
    pos_embed = get_2d_sincos_pos_embed_from_grid(embed_dim, grid)  # (H*W, D)
    if cls_token:
        pos_embed = np.concatenate([np.zeros([1, embed_dim]), pos_embed], axis=0)  # (1+H*W, D)
    return pos_embed


class MViT(nn.Module):
    def __init__(self, in_chans=1, bands=30, num_classes=9, dim=64, heads=4, depth=3, dropout=0.2):
        super(MViT, self).__init__()
        self.conv3d = nn.Sequential(nn.Conv3d(1, 8, 3), nn.BatchNorm3d(8), nn.ReLU())
        self.conv2d = nn.Sequential(nn.Conv2d(8*28, 64, 3), nn.BatchNorm2d(64), nn.ReLU())

        self.cls_token = nn.Parameter(torch.zeros(1, 1, dim))
        nn.init.normal_(self.cls_token, std=.02)

        self.pos_embed = nn.Parameter(torch.zeros(1, 121 + 1, dim), requires_grad=False)
        pos_embed = get_2d_sincos_pos_embed(dim, 11, cls_token=True)
        self.pos_embed.data.copy_(torch.from_numpy(pos_embed).float().unsqueeze(0))

        self.blocks = nn.ModuleList([Block(dim, heads, qkv_bias=True, attn_drop=0.1) for _ in range(depth)])

        self.norm = nn.LayerNorm(dim)

        self.cls_head = nn.Linear(dim, num_classes)
        self.dropout = nn.Dropout(dropout)

    def random_masking(self, x, mask_ratio=0.75):
        N, L, D = x.shape  # batch, length, dim
        len_keep = int(L * (1 - mask_ratio))
        noise = torch.rand(N, L, device=x.device)  # noise in [0, 1]  (N, L)
        # sort noise for each sample
        ids_shuffle = torch.argsort(noise, dim=1)  # 从小到大排序，返回索引 (N, L)
        # keep the first subset
        ids_keep = ids_shuffle[:, :len_keep]
        x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))
        return x_masked

    def forward(self, x, mask_ratio=0.75):
        # (B, 1, 30, 15, 15)
        x = self.conv3d(x)  # (B, 8, 28, 13, 13)
        x = rearrange(x, 'b c d h w -> b (c d) h w')  # (B, 224, 13, 13)
        x = self.conv2d(x)  # (B, 64, 11, 11)
        x = rearrange(x, 'b c h w -> b (h w) c')  # (B, 121, 64)

        x = x + self.pos_embed[:, 1:, :]

        center_embed = x[:, 60, :].unsqueeze(1)  # (B, 1, 64)
        x = torch.cat([x[:, :60, :], x[:, 61:, :]], dim=1)  # (B, 120, 64)

        if self.training:
            x = self.random_masking(x, mask_ratio)  # (B, 30, 64)

        x = torch.cat([center_embed, x], dim=1)

        cls_token = self.cls_token + self.pos_embed[:, :1, :]
        cls_token = repeat(cls_token, '1 n d -> b n d', b = x.shape[0])  # (B, 1, 64)

        x = torch.cat((cls_token, x), dim = 1)

        x = self.dropout(x)  # (B, 31, 64)

        for blk in self.blocks:
            x = blk(x)

        latent = self.norm(x[:, 0, :])
        x = self.cls_head(latent)
        return x

In [32]:
# warning deleted ` drop=0.1` from `Block` note

In [33]:
from torch import optim

model = MViT(num_classes=9).cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

In [36]:
from MViT.CNNUtils import train, test, valid
import time

print('start training')
acc_list = [0.00]

tic = time.time()
for epoch in range(2):
    # print each batch input
    for batch_idx, (batch_x, batch_y) in enumerate(train_loader):
        print("Epoch: {} | Batch: {} | x shape: {} | y shape: {}".format(
            epoch+1, batch_idx+1, batch_x.shape, batch_y.shape))
        print("x min: {:.4f} | x max: {:.4f}".format(batch_x.min(), batch_x.max()))
        print("labels in this batch: {}".format(batch_y))
        print("--------------------------------------------------")

    # then actually train
    train_acc, train_loss = train(model, train_loader, criterion, optimizer)
    print("Epoch: {:03d} - train_loss: {:.4f} - train_acc: {:.4f}".\
          format(epoch+1, train_loss, train_acc))
    scheduler.step()

toc = time.time()
print("Running Time: {:.2f}".format(toc-tic))
print("**************************************************")

Streaming output truncated to the last 5000 lines.
        1, 7, 0, 1, 0, 1])
--------------------------------------------------
Epoch: 2 | Batch: 428 | x shape: torch.Size([30, 1, 30, 15, 15]) | y shape: torch.Size([30])
x min: -8.1010 | x max: 9.3981
labels in this batch: tensor([6, 1, 1, 1, 1, 3, 8, 3, 6, 1, 1, 5, 1, 0, 0, 1, 3, 1, 0, 1, 1, 1, 0, 3,
        1, 1, 0, 4, 1, 1])
--------------------------------------------------
Epoch: 2 | Batch: 429 | x shape: torch.Size([30, 1, 30, 15, 15]) | y shape: torch.Size([30])
x min: -7.9793 | x max: 7.5681
labels in this batch: tensor([0, 5, 1, 1, 1, 0, 1, 1, 1, 8, 1, 1, 7, 1, 1, 1, 2, 0, 3, 3, 3, 1, 1, 0,
        0, 1, 5, 1, 1, 0])
--------------------------------------------------
Epoch: 2 | Batch: 430 | x shape: torch.Size([30, 1, 30, 15, 15]) | y shape: torch.Size([30])
x min: -8.3029 | x max: 7.4680
labels in this batch: tensor([5, 1, 6, 1, 1, 1, 5, 7, 5, 0, 1, 6, 1, 8, 5, 8, 1, 1, 5, 3, 3, 0, 1, 0,
        1, 4, 1, 4, 1, 3])
----------